In [1]:
import os
import time
import signal
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from contextlib import contextmanager
import numpy as np
from TDD.TDD import Ini_TDD,Clear_TDD,set_index_order
from TDD.TDD_Q import cir_2_tn,add_trace_line,add_inputs,add_outputs
#get_real_qubit_num
from TDD.TN import Index,Tensor,TensorNetwork

In [2]:


# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)
def count_cnot_gates(circuit):
    # Count the number of 2-qubit CNOT gates
    num_cnot_gates = sum(1 for instr, _, _ in circuit.data if instr.name == 'cx')
    return num_cnot_gates

def benchmark_circuits(directory,aer_method="statevector"):
    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open(aer_method+"_benchmark.txt", "w")
    results_file.write("Circuit Name,Time (s),Number of Qubits,Number of Gates,Number of CNOT Gates,Transpile Time\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

            # Transpile the circuit for simulation
            backend = AerSimulator(method=aer_method)
            start_time = time.time()
            qc=transpile(circuit,backend=backend)
            end_time = time.time()
            transpile_time = end_time - start_time

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_cnot_gates = count_cnot_gates(circuit)

            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                job=backend.run(qc)
                result=job.result()
                end_time = time.time()
                runtime = end_time - start_time

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{num_qubits},{num_gates},{num_cnot_gates},{transpile_time}\n")

        except TimeoutException:
            # If the simulation times out, write "TO" as runtime
            results_file.write(f"{qasm_file},TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
benchmark_circuits(directory_path,"matrix_product_state")


Error processing bv_70.qasm: 'Number of qubits (70) in circuit-200 is greater than maximum (63) in the coupling_map'
Error processing bv_80.qasm: 'Number of qubits (80) in circuit-249 is greater than maximum (63) in the coupling_map'
Error processing bv_100.qasm: 'Number of qubits (100) in circuit-271 is greater than maximum (63) in the coupling_map'
Error processing bv_90.qasm: 'Number of qubits (90) in circuit-286 is greater than maximum (63) in the coupling_map'


In [3]:


# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)
def count_cnot_gates(circuit):
    # Count the number of 2-qubit CNOT gates
    num_cnot_gates = sum(1 for instr, _, _ in circuit.data if instr.name == 'cx')
    return num_cnot_gates

def benchmark_circuits(directory,aer_method="Matrix_product_state"):
    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open("TDD_benchmark.txt", "w")
    results_file.write("Circuit Name,Time (s),Number of Qubits,Number of Gates,Number of CNOT Gates,Initialisation Time\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_cnot_gates = count_cnot_gates(circuit)


            # create the tensor network
            start_time = time.time()
            input_s=[0]*num_qubits
            tn,all_indexs=cir_2_tn(circuit)
            add_inputs(tn,input_s,num_qubits)
            initialisation_time=time.time()-start_time
            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                Ini_TDD(index_order=all_indexs)
                tdd=tn.cont(optimizer='tree_decomposition')
                end_time = time.time()
                runtime = end_time - start_time

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{num_qubits},{num_gates},{num_cnot_gates},{initialisation_time}\n")

        except TimeoutException:
            # If the simulation times out, write "TO" as runtime
            results_file.write(f"{qasm_file},TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
benchmark_circuits(directory_path,"automatic")
